In [ ]:
# 01_exploratory_analysis.ipynb

# 📊 Análise Exploratória: Jena Climate Dataset

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dtaidistance import dtw
import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram

# --- 1. Carregar o dataset ---
df = pd.read_csv('data/jena_climate_2009_2016.csv')
df['Date Time'] = pd.to_datetime(df['Date Time'], format="%d.%m.%Y %H:%M:%S", dayfirst=True)

# --- 2. Amostra e estrutura ---
print(df.shape)
df.info()
df.head()

# --- 3. Verificar valores ausentes ---
missing = df.isna().sum()
print("\nValores ausentes por coluna:\n", missing[missing > 0])

# --- 4. Estatísticas descritivas ---
df.describe().T

# --- 5. Amostragem horária com interpolacao ---
df_hourly = df.resample('H', on='Date Time').mean().interpolate()

# --- 6. Visualização de temperatura ao longo do tempo ---
plt.figure(figsize=(15, 4))
plt.plot(df_hourly.index, df_hourly['T (degC)'], label='Temperatura (C)')
plt.title('Temperatura ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('Temperatura (°C)')
plt.grid(True)
plt.tight_layout()
plt.show()

# --- 7. Correlações entre variáveis ---
plt.figure(figsize=(12, 10))
sns.heatmap(df_hourly.corr(), cmap='coolwarm', annot=False, center=0)
plt.title('Matriz de Correlação entre Variáveis')
plt.tight_layout()
plt.show()

# --- 8. Distribuições individuais ---
variaveis = [
    'T (degC)', 'p (mbar)', 'Tpot (K)', 'rh (%)', 'wv (m/s)', 'max. wv (m/s)',
    'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 
    'H2OC (mmol/mol)', 'rho (g/m**3)', 'wd (deg)'
]

plt.figure(figsize=(18, 20))
for i, var in enumerate(variaveis, 1):
    plt.subplot(5, 3, i)
    sns.histplot(df_hourly[var], kde=True, bins=40, color='skyblue')
    plt.title(f'Distribuição de {var}')
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()

# --- 9. Matriz de Distância DTW entre variáveis ---
# Subsample (ex: 1000 registros) para otimizar tempo
df_sample = df_hourly[variaveis].dropna().iloc[:1000]

# Criar matriz de distância DTW
dtw_matrix = np.zeros((len(variaveis), len(variaveis)))

for i, var1 in enumerate(variaveis):
    for j, var2 in enumerate(variaveis):
        if i < j:
            dtw_matrix[i, j] = dtw.distance(df_sample[var1].values, df_sample[var2].values)
            dtw_matrix[j, i] = dtw_matrix[i, j]  # simétrica

# Visualizar com seaborn
plt.figure(figsize=(12, 10))
sns.heatmap(dtw_matrix, xticklabels=variaveis, yticklabels=variaveis,
            cmap="viridis", annot=True, fmt=".1f")
plt.title("Matriz de Distância DTW entre Variáveis Climáticas")
plt.tight_layout()
plt.show()

# --- 10. Dendrograma: Clusterização hierárquica das variáveis ---
dist_vec = squareform(dtw_matrix, checks=False)
linkage_matrix = linkage(dist_vec, method='ward')

plt.figure(figsize=(14, 6))
dendrogram(linkage_matrix, labels=variaveis, leaf_rotation=90)
plt.title("Dendrograma: Clusterização das Variáveis com Base na Distância DTW")
plt.tight_layout()
plt.show()